In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, '../raw')
# sys.path.clear()

In [3]:
import json, csv
import numpy as np
import pandas as pd

TRAIN_PATH = '../raw/train_data_processed.csv'

In [4]:
# train_csv = list(csv.reader(open(TRAIN_PATH)))
# train_csv

train_df = pd.read_csv(TRAIN_PATH)
train_df[20:40:2]

,id,keyword,location,text,hashtag,subj,verb,obj,mentioned_news_org,mentioned_relief_org,orgs,gpes,facs,target
20,31,NaN,NaN,this is ridiculous....,[],[],[],[],False,False,[],[],[],0
22,33,NaN,NaN,Love skiing,[],[],[],[],False,False,[],[],[],0
24,36,NaN,NaN,LOOOOOOL,[],[],[],[],False,False,[],[],[],0
26,38,NaN,NaN,Was in NYC last week!,[],[],[],[],False,False,['NYC'],[],[],0
28,40,NaN,NaN,Cooool :),[],[],[],[],False,False,[],[],[],0
30,44,NaN,NaN,The end!,[],[],[],[],False,False,[],[],[],0
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,"['#metal', '#RT']",[],"['try', 'bring']",[],False,False,[],[],[],0
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,[],[],"['set', 'cry']",[],False,False,[],[],[],0
36,54,ablaze,Pretoria,@PhDSquares #mufc they've built so much hype a...,['#mufc'],[],"['doubt', 'set', 'build']",[],False,False,['EPL'],[],[],0
38,56,ablaze,NaN,Barbados #Bridgetown JAMAICA ÛÒ Two cars set ...,['#Bridgetown'],"['barbados', 'car']",['set'],[],False,False,['the St Elizabeth Police Superintende'],[],[],1


## handle missing values in a naive way

In [5]:
fill_v = {'keyword': 'missing', 'location': 'unknown'}
filled_df = train_df.fillna(value=fill_v)
filled_df.head()

,id,keyword,location,text,hashtag,subj,verb,obj,mentioned_news_org,mentioned_relief_org,orgs,gpes,facs,target
0,1,missing,unknown,Our Deeds are the Reason of this #earthquake M...,['#earthquake'],"['deed', 'allah']",['forgive'],[],False,False,[],[],[],1
1,4,missing,unknown,Forest fire near La Ronge Sask. Canada,[],[],[],[],False,False,[],['Canada'],['La Ronge Sask'],1
2,5,missing,unknown,All residents asked to 'shelter in place' are ...,[],"['evacuation', 'resident']","['expect', 'ask', 'notify', 'shelter']",[],False,False,"['shelter', 'officers', 'other', 'shelter', 'o...",[],[],1
3,6,missing,unknown,"13,000 people receive #wildfires evacuation or...",['#wildfires'],['people'],['receive'],[],False,False,['orders'],['California'],[],1
4,7,missing,unknown,Just got sent this photo from Ruby #Alaska as ...,"['#Alaska', '#wildfires']",[],"['send', 'get']",[],False,False,[],[],[],1


In [6]:
X = filled_df.drop(['target'], axis=1)
y = filled_df['target']
X.shape

(7613, 13)

## simple linear SVC - With Hashtag and POS 


In [7]:
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer

class LabeledNormalizer(Normalizer):
    def fit(self, X, *args, **kwargs):
        try:
            self.names = X.columns
        except:
            self.names = [str(i) for i in range(X.shape[1])]
        return super().fit(X, *args, **kwargs)
        
    def get_feature_names(self):
        return self.names
    
vec = ColumnTransformer([
    ('norm', LabeledNormalizer(), ['id']),
    ('kw', TfidfVectorizer(ngram_range=(1, 3), min_df=2, token_pattern=r"(?u)\b\w+\b",), 'keyword'),
    ('loc', TfidfVectorizer(ngram_range=(1, 3), min_df=2, token_pattern=r"(?u)\b\w+\b"), 'location'),
    ('text', TfidfVectorizer(ngram_range=(1, 3), min_df=2, token_pattern=r"(?u)\b\w+\b"), 'text'),
    ('hashtag', TfidfVectorizer(ngram_range=(1, 3), min_df=2, token_pattern=r"\[(?u)\b\w+\b\]"), 'hashtag'),
    ('subj', TfidfVectorizer(ngram_range=(1, 3), min_df=2, token_pattern=r"\[(?u)\b\w+\b\]"), 'subj'),
    ('verb', TfidfVectorizer(ngram_range=(1, 3), min_df=2, token_pattern=r"\[(?u)\b\w+\b\]"), 'verb'),
    ('obj', TfidfVectorizer(ngram_range=(1, 3), min_df=2, token_pattern=r"\[(?u)\b\w+\b\]"), 'obj'),
])

from sklearn.svm import LinearSVC
from sklearn.svm import SVC
clf = SVC(kernel='linear', probability=True, verbose=True)
# clf = LinearSVC(verbose=True)


pipeline = make_pipeline(vec, clf)

## evaluation

In [8]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict

def evaluate(_clf, X, y):
    report = classification_report(
        y_true=y, y_pred=cross_val_predict(pipeline, X, y, cv=5)
    )
    print(report)
    scores = cross_val_score(_clf, X, y, scoring='accuracy', cv=10)
    print('Cross-validation Accuracy: {:.3f} ± {:.3f}'.format(np.mean(scores), 2 * np.std(scores)))
    
    _clf.fit(X,y)
    print('Training Set Accuracy: {:.3f}'.format(_clf.score(X,y)))


evaluate(pipeline, X, y)

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
from sklearn.externals import joblib
import os

pipeline.fit(X,y)
baseline_mdl = '../models/baseline-linear-svc.pkl'
joblib.dump(pipeline, baseline_mdl)
!ls -lSh $baseline_mdl